In [41]:
# 필요한 라이브러리 임포트
import pandas as pd
import openai
import json
import gc
import torch
from datetime import datetime
from zoneinfo import ZoneInfo
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import ElasticVectorSearch
from langchain.chat_models import ChatOpenAI
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore


In [42]:
# .env 파일에서 환경 변수 로드
load_dotenv("/data/ephemeral/home/upstage-ai-advanced-ir2-private/OPENAI_API_KEY.env")

# OpenAI API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# API 키 설정
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    raise ValueError("API key not found. Please check your .env file.")

In [43]:
es_username = "elastic"
es_password = "cqMxV1GNi7I1*MU9wCr*"

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/elasticsearch-8.15.2/config/certs/http_ca.crt") # TLS Error시 ca_certs의 경로 변경하기

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-12587', 'cluster_name': 'elasticsearch', 'cluster_uuid': '3PxwSecZTmCW-m9du-USxA', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [ ]:
#------>>>새로운 문서 및 임베딩 모델 사용 시 주석 풀고 사용하면됨

index_name = 'langchain_index_test'

# 임베딩 함수 생성
embedding = OpenAIEmbeddings() # ("text-embedding-ada-002") # text-embedding-3-large # model="text-embedding-ada-002"

# ElasticsearchStore 설정
vector_store = ElasticsearchStore(
    #elasticsearch_url="https://localhost:9200",
    index_name=index_name,
    embedding=embedding,
    es_connection=es
)

# Elasticsearch 연결 확인
try:
    response = vector_store.client.info()
    print("Elasticsearch 연결 성공:", response)
except Exception as e:
    print("Elasticsearch 연결 실패:", e)

Elasticsearch 연결 성공: {'name': 'instance-12587', 'cluster_name': 'elasticsearch', 'cluster_uuid': '3PxwSecZTmCW-m9du-USxA', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


/tmp/ipykernel_783092/3000926933.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model="text-embedding-ada-002") # ("text-embedding-ada-002") # text-embedding-3-large
/tmp/ipykernel_783092/3000926933.py:9: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use :class:`~Use class in langchain-elasticsearch package` instead.
  vector_store = ElasticsearchStore(


In [5]:
# ------>>>새로운 문서 및 임베딩 모델 사용 시 주석 풀고 사용하면됨

# 문서 데이터 로드 및 인덱싱
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
import glob

def load_documents(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            json_obj = json.loads(line)
            content = json_obj.get('content', '')
            docid = json_obj.get('docid', '')
            doc = Document(page_content=content, metadata={'docid': docid})
            documents.append(doc)
    return documents

# 문서 데이터 로드
documents = load_documents('/data/ephemeral/home/upstage-ai-advanced-ir2-private/data/chunked_documents.jsonl')  # 실제 문서 경로로 변경하세요



In [6]:
# ------>>>새로운 문서 및 임베딩 모델 사용 시 주석 풀고 사용하면됨

# 임베딩을 저장할 리스트 초기화
embeddings_data = []

# 문서 데이터 인덱싱 및 임베딩 저장
for doc in documents:
    # 임베딩 생성
    embedding_vector = embedding.embed_query(doc.page_content)  # OpenAI 임베딩 생성
#    print(f"Doc ID: {doc.metadata['docid']}, Embedding: {embedding_vector}")  # 임베딩 출력
    
    # 문서와 임베딩을 리스트로 저장
    embeddings_data.append({
        'docid': doc.metadata['docid'],
        'embedding': embedding_vector,  # 이미 리스트 형태
        'content': doc.page_content  # 원본 문서 내용 추가
    })
print("리스트 저장완료")

In [6]:
from tqdm import tqdm
import time

start_time = time.time()
total_docs = len(documents)

for i, doc in enumerate(tqdm(documents, desc="문서 인덱싱 중")):
    vector_store.add_documents([doc])
    
    if (i + 1) % 100 == 0:  # 100개 문서마다 상태 출력
        elapsed = time.time() - start_time
        docs_per_sec = (i + 1) / elapsed
        print(f"\n처리된 문서: {i+1}/{total_docs}")
        print(f"평균 처리 속도: {docs_per_sec:.2f} 문서/초")

print(f"\n총 소요 시간: {time.time() - start_time:.2f}초")
print("인덱싱 완료")

문서 인덱싱 중:   2%|▏         | 100/6500 [00:41<34:42,  3.07it/s] 


처리된 문서: 100/6500
평균 처리 속도: 2.39 문서/초


문서 인덱싱 중:   3%|▎         | 200/6500 [01:23<37:04,  2.83it/s]  


처리된 문서: 200/6500
평균 처리 속도: 2.40 문서/초


문서 인덱싱 중:   5%|▍         | 300/6500 [02:06<56:05,  1.84it/s]  

...


처리된 문서: 6300/6500
평균 처리 속도: 1.65 문서/초


문서 인덱싱 중:  98%|█████████▊| 6400/6500 [1:04:27<00:52,  1.90it/s]


처리된 문서: 6400/6500
평균 처리 속도: 1.65 문서/초


문서 인덱싱 중: 100%|██████████| 6500/6500 [1:05:16<00:00,  1.66it/s]


처리된 문서: 6500/6500
평균 처리 속도: 1.66 문서/초

총 소요 시간: 3916.97초
인덱싱 완료


In [ ]:
  
embeddings_data 출력
print(f"Current embeddings_data: {embeddings_data}")  # 추가된 데이터 출력

Elasticsearch에 문서 데이터 인덱싱
vector_store.add_documents(documents)  # 여기서 documents를 인덱싱
print("인덱싱 완료")

임베딩을 JSON 파일로 저장
with open('/data/ephemeral/home/upstage-ai-advanced-ir2-private/elasticsearch/embeddings.json', 'w', encoding='utf-8') as f:
    json.dump(embeddings_data, f, ensure_ascii=False, indent=4)

print("임베딩이 embeddings.json 파일에 저장되었습니다.")



### 저장해둔 인덱스 있으면 바로 실행하면 됨

In [44]:
# Elasticsearch 클라이언트 사용하여 문서 수 확인
count_response = vector_store.client.count(index=index_name)
print(f"인덱스 '{index_name}'에 있는 문서 수: {count_response['count']}")


인덱스 'langchain_index_test'에 있는 문서 수: 6500


In [45]:

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from pydantic import BaseModel, Field
from typing import List

# 1. Pydantic 모델 정의
class ScienceQuery(BaseModel):
    main_query: str = Field(description="주요 검색 쿼리")
    expanded_queries: List[str] = Field(description="관련 검색 쿼리들 (최대 3개)")
    category: str = Field(description="과학 카테고리 (물리/화학/생물/지구과학/천문학/기타)")
    subtopic: str = Field(description="세부 주제")
    is_scientific: bool = Field(description="과학 관련 질문 여부")

# 2. Output Parser 설정
science_parser = PydanticOutputParser(pydantic_object=ScienceQuery)

# 3. 프롬프트 템플릿 수정
query_template = """
과학 질문을 분석하여 효과적인 검색 쿼리를 생성하세요.

{format_instructions}

대화 기록:
{chat_history}
"""

query_prompt = ChatPromptTemplate.from_template(
    template=query_template,
    partial_variables={"format_instructions": science_parser.get_format_instructions()}
)

# 4. LLM Chain 설정
query_chain = LLMChain(
    llm=llm,
    prompt=query_prompt,
    output_parser=science_parser
)

# 5. 컨텍스트 압축 검색기
def get_compression_retriever(vector_store):
    base_retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    
    compressor = LLMChainExtractor.from_llm(llm)
    
    return ContextualCompressionRetriever(
        base_retriever=base_retriever,
        base_compressor=compressor
    )

# 6. 검색 함수
def search(query: ScienceQuery, compression_retriever, max_k=3):
    """개선된 벡터 유사도 검색"""
    try:
        # 메인 쿼리로 검색
        main_docs = compression_retriever.get_relevant_documents(query.main_query)
        results = []
        
        # 확장 쿼리로 추가 검색
        for expanded_query in query.expanded_queries:
            additional_docs = compression_retriever.get_relevant_documents(expanded_query)
            main_docs.extend(additional_docs)
        
        # 결과 처리 및 중복 제거
        seen_ids = set()
        for doc in main_docs:
            docid = doc.metadata.get('docid', '')
            if docid not in seen_ids and len(results) < max_k:
                seen_ids.add(docid)
                results.append({
                    'docid': docid,
                    'score': doc.metadata.get('score', 0.0),
                    'content': doc.page_content
                })
        
        return results
    except Exception as e:
        print(f"Search failed: {e}")
        return []

# 7. 메인 처리 함수
def handle_query(chat_history, compression_retriever=None, references=None):
    if references is None:
        try:
            # 과학 쿼리 생성
            query_result = query_chain.run(chat_history=chat_history)
            
            # 검색 수행
            if compression_retriever:
                search_results = search(query_result, compression_retriever)
                return query_result.main_query, search_results
            return query_result.main_query, []
            
        except Exception as e:
            print(f"Error in handle_query: {e}")
            return None, []
    else:
        try:
            answer = qa_chain.run(chat_history=chat_history, references=references)
            return answer
        except Exception as e:
            print(f"Error generating answer: {e}")
            return "답변 생성 중 오류가 발생했습니다."

# 8. 평가 파일 처리 함수
def process_eval_file(eval_filename, vector_store, use_pre_query=False, max_k=3):
    current_time = datetime.now(ZoneInfo("Asia/Seoul")).strftime("%m%d-%H%M")
    output_filename = f"/data/ephemeral/home/upstage-ai-advanced-ir2-private/data_langchain/{current_time}_topk{max_k}.csv"
    
    compression_retriever = get_compression_retriever(vector_store)
    results = []

    with open(eval_filename, "r", encoding='utf-8') as f:
        for line in f:
            j = json.loads(line)
            if "msg" not in j:
                continue

            chat_history = j["msg"]
            
            # 쿼리 생성 및 검색
            standalone_query, search_results = handle_query(
                chat_history, 
                compression_retriever
            )

            if standalone_query and search_results:
                topk_results = search_results[:max_k]
                references = [{'score': res['score'], 'content': res['content']} 
                            for res in topk_results]
                references_text = '\n\n'.join([ref['content'] for ref in references])
                answer = handle_query(chat_history, None, references_text)
            else:
                answer = "정보가 부족해서 답을 할 수 없습니다."
                references = []
                topk_results = []

            results.append({
                "eval_id": j.get("eval_id"),
                "standalone_query": standalone_query,
                "topk": [res['docid'] for res in topk_results] if topk_results else [],
                "answer": answer,
                "references": references
            })

            # 메모리 관리
            gc.collect()
            torch.cuda.empty_cache()

    # 결과 저장
    with open(output_filename, 'w', encoding='utf-8') as f_out:
        for result in results:
            f_out.write(json.dumps(result, ensure_ascii=False) + '\n')
            
    print(f"결과가 {output_filename}에 저장되었습니다.")


In [46]:
eval_filename = "/data/ephemeral/home/upstage-ai-advanced-ir2-private/data/eval_copy.jsonl"
process_eval_file(eval_filename, vector_store)

KeyboardInterrupt: 

In [32]:
# # 메인 실행 부분
# if __name__ == "__main__":
#     eval_file_path = '/data/ephemeral/home/upstage-ai-advanced-ir2-private/data/eval_copy.jsonl'

#     # 평가 파일 처리 및 저장 함수 호출
#     process_eval_file(eval_file_path, use_pre_query=True, max_k=3)



Processing chat history: [{'role': 'user', 'content': '공에 힘이 주어졌을 때 공이 어떻게 움직이는지 과학적으로 설명해줘.'}]
Generated message: content='' additional_kwargs={'function_call': {'arguments': '{"standalone_query":"공에 힘이 주어졌을 때 움직임"}', 'name': 'search'}} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 199, 'total_tokens': 222, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-0963c0ab-794d-4ec8-bbc5-742248a3d3e2-0'
Generated standalone query: 공에 힘이 주어졌을 때 움직임
Searching for: 공에 힘이 주어졌을 때 움직임
Generated answer: ```json
{
  "query": "공에 힘이 주어졌을 때 움직임의 과학적 원리",
  "answer": ""
}
```
Processed result: {'eval_id': 223, 'standalone_query': '공에 힘이 주어졌을 때 움직임', 'topk': ['e2161953-e80b-41b3-a83b-5bc7d76a801c', '83f6c921-1ee7-4bd5-a124-7c71c860546a', '88e701bc-b7cf-4c1e-849a-e21